In [37]:
from keras.models import Sequential , Model
from keras.layers import Dense , Conv2D , MaxPool2D , BatchNormalization , Reshape , Input , Flatten
from keras.layers.advanced_activations import LeakyReLU

from keras.optimizers import Adam

from keras.datasets import mnist

In [38]:
import os

import matplotlib as plt
import numpy as np

%matplotlib inline

In [39]:
WIDTH = 96
HEIGHT = 96
CHANNEL = 3

LATENT_DIM = 100 #latent variable z

BATCH_SIZE = 128
EPOCHS = 10

PATH = 'faces/'

#生成多少个图像 长*宽
ROW = 5
COL = 5

In [40]:
load_index = 0

images_name = os.listdir(PATH)

IMAGES_COUNT = len(images_name)

In [41]:
'''
(X_train , y_train),(X_test , y_test) = mnist.load_data()
X_train = X_train/127.5-1
X_train = np.expand_dims(X_train , 3)
'''

'\n(X_train , y_train),(X_test , y_test) = mnist.load_data()\nX_train = X_train/127.5-1\nX_train = np.expand_dims(X_train , 3)\n'

In [42]:
'''
def load_mnist():
    return X_train[np.random.randint(0, X_train.shape[0], BATCH_SIZE)]
    
def write_image_mnist(epoch):
    
    noise = np.random.normal(size = (ROW*COL , LATENT_DIM))
    generated_image = generator_i.predict(noise)
    generated_image = generated_image*0.5+0.5
    
    fig , axes = plt.pyplot.subplots(ROW , COL)
    
    count=0
    
    for i in range(ROW):
        for j in range(COL):
            axes[i][j].imshow(generated_image[count,:,:,0] , cmap = 'gray')
            axes[i][j].axis('off')
            count += 1
            
    fig.savefig('images/No.%d.png' % epoch)
    plt.pyplot.close()

'''

"\ndef load_mnist():\n    return X_train[np.random.randint(0, X_train.shape[0], BATCH_SIZE)]\n    \ndef write_image_mnist(epoch):\n    \n    noise = np.random.normal(size = (ROW*COL , LATENT_DIM))\n    generated_image = generator_i.predict(noise)\n    generated_image = generated_image*0.5+0.5\n    \n    fig , axes = plt.pyplot.subplots(ROW , COL)\n    \n    count=0\n    \n    for i in range(ROW):\n        for j in range(COL):\n            axes[i][j].imshow(generated_image[count,:,:,0] , cmap = 'gray')\n            axes[i][j].axis('off')\n            count += 1\n            \n    fig.savefig('images/No.%d.png' % epoch)\n    plt.pyplot.close()\n\n"

In [43]:
def load_image(batch_size = BATCH_SIZE):
    global load_index
    
    images = []
    
    for i in range(batch_size):
        images.append(plt.image.imread(PATH + images_name[(load_index + i) % IMAGES_COUNT]))
    
    load_index += batch_size
    
    return np.array(images)/127.5-1

def write_image(epoch):
    
    noise = np.random.normal(size = (ROW*COL , LATENT_DIM))
    generated_image = generator_i.predict(noise)
    generated_image = (generated_image+1)*127.5
    
    fig , axes = plt.pyplot.subplots(ROW , COL)
    
    count=0
    
    for i in range(ROW):
        for j in range(COL):
            axes[i][j].imshow(generated_image[count])
            axes[i][j].axis('off')
            count += 1
            
    fig.savefig('generated_faces/No.%d.png' % epoch)
    plt.pyplot.close()
    
    #plt.image.imsave('images/'+str(epoch)+'.jpg')

In [44]:
def generator():
    #sample from noise z
    model = Sequential(name='generator')
    
    model.add(Dense(units = 256 , input_dim = LATENT_DIM , name='dense1'))
    model.add(LeakyReLU(0.2))
    model.add(BatchNormalization(momentum=0.8 , name='batchnorm1'))
    model.add(Dense(units = 512 , name='dense2'))
    model.add(LeakyReLU(0.2))
    model.add(BatchNormalization(momentum=0.8 , name='batchnorm2'))
    model.add(Dense(units = 1024 , name='dense3'))
    model.add(LeakyReLU(0.2))
    model.add(BatchNormalization(momentum=0.8 , name='batchnorm3'))
    #model.add(Dense(units = 5000 , activation='relu' , name='dense4'))
    #model.add(BatchNormalization(name='batchnorm4'))
    #model.add(Dense(units = 10000 , activation='relu' , name='dense5'))
    #model.add(BatchNormalization(name='batchnorm5'))
    #model.add(Dense(units = 20000 , activation='relu' , name='dense6'))
    #model.add(BatchNormalization(name='batchnorm6'))
    model.add(Dense(units = WIDTH*HEIGHT*CHANNEL , activation='tanh' , name='dense7'))
    model.add(Reshape(target_shape=(WIDTH , HEIGHT , CHANNEL) , name='reshape1'))
    
    #model.summary()
    
    noise = Input(shape=(LATENT_DIM , ) , name='input1')
    image = model(noise)
    
    return Model(noise , image , name='generator_Model')

In [45]:
def discriminator():
    #input a image to discriminate real or fake
    model = Sequential(name='discriminator')
    
    model.add(Flatten(input_shape = (WIDTH,HEIGHT,CHANNEL) , name='flatten1'))
    #model.add(Dense(units=10000 , name='dense8'))
    #model.add(LeakyReLU(0.3 , name='leakyrelu1'))
    #model.add(Dense(units=5000 , name='dense9'))
    #model.add(LeakyReLU(0.3 , name='leakyrelu2'))
    #model.add(Dense(units=1024 , name='dense10'))
    #model.add(LeakyReLU(0.3 , name='leakyrelu3'))
    model.add(Dense(units=512 , name='dense11'))
    model.add(LeakyReLU(0.2 , name='leakyrelu4'))
    model.add(Dense(units=256 , name='dense12'))
    model.add(LeakyReLU(0.2 , name='leakyrelu5'))
    #model.add(Dense(units=32 , name='dense13'))
    #model.add(LeakyReLU(0.3 , name='leakyrelu6'))
    #model.add(Dense(units=4 , name='dense14'))
    #model.add(LeakyReLU(0.3 , name='leakyrelu7'))
    model.add(Dense(units=1 , activation='sigmoid' , name='dense15'))
    
    #model.summary()
    
    image = Input(shape=(WIDTH , HEIGHT , CHANNEL) , name='input1')
    validity = model(image)
    
    return Model(image , validity , name='discriminator_Model')

In [46]:
def combined_model(generator_i , discriminator_i):
    #生成器和判别器组合成整体
    z = Input(shape=(LATENT_DIM , ) , name='z')
    
    image = generator_i(z)
    discriminator_i.trainable = False
    validity = discriminator_i(image)
    
    return Model(z , validity , name='combined_model')

In [47]:
discriminator_i = discriminator()
discriminator_i.compile(optimizer=Adam(lr=0.0002 , beta_1=0.5) , loss='binary_crossentropy' , metrics=['accuracy'])

generator_i = generator()

combined_model_i = combined_model(generator_i , discriminator_i)


combined_model_i.compile(optimizer=Adam(lr=0.0002 , beta_1=0.5) , loss='binary_crossentropy')

In [ ]:
real_labels = np.ones(shape=(BATCH_SIZE , 1)) #真实样本label为1
fake_labels = np.zeros(shape=(BATCH_SIZE , 1)) #假样本label为0

for i in range(20000):
    noise = np.random.normal(size=(BATCH_SIZE , LATENT_DIM))

    real_image = load_image()
    #real_image = load_mnist()
    #训练判别器
    fake_image = generator_i.predict(noise)

    real_loss = discriminator_i.train_on_batch(real_image , real_labels)
    fake_loss = discriminator_i.train_on_batch(fake_image , fake_labels)

    loss = np.add(real_loss , fake_loss)/2

    #训练生成器
    noise2 = np.random.normal(size=(BATCH_SIZE , LATENT_DIM))
    generator_loss = combined_model_i.train_on_batch(noise2 , real_labels)
    
    print('epoch:%d loss:%f accu:%f gene_loss:%f' % (i , loss[0] , loss[1] , generator_loss))
    
    if i % 50 == 0:
        write_image(i)
        #write_image_mnist(i)

C:\Anaconda3\lib\site-packages\keras\engine\training.py:478: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch:0 loss:0.740570 accu:0.296875 gene_loss:0.702766
epoch:1 loss:0.537559 accu:0.542969 gene_loss:0.555938
epoch:2 loss:0.675689 accu:0.523438 gene_loss:0.530261
epoch:3 loss:0.672871 accu:0.542969 gene_loss:0.534743
epoch:4 loss:0.695099 accu:0.546875 gene_loss:0.521769
epoch:5 loss:0.628405 accu:0.574219 gene_loss:0.590071
epoch:6 loss:0.575869 accu:0.609375 gene_loss:0.690930
epoch:7 loss:0.571797 accu:0.640625 gene_loss:0.758851
epoch:8 loss:0.533622 accu:0.636719 gene_loss:0.871782
epoch:9 loss:0.452216 accu:0.687500 gene_loss:0.987085
epoch:10 loss:0.450965 accu:0.710938 gene_loss:1.161847
epoch:11 loss:0.347822 accu:0.785156 gene_loss:1.293101
epoch:12 loss:0.311907 accu:0.828125 gene_loss:1.445907
epoch:13 loss:0.312792 accu:0.808594 gene_loss:1.497506
epoch:14 loss:0.247289 accu:0.890625 gene_loss:1.524854
epoch:15 loss:0.225497 accu:0.898438 gene_loss:1.610508
epoch:16 loss:0.248967 accu:0.917969 gene_loss:1.653491
epoch:17 loss:0.350959 accu:0.832031 gene_loss:1.715170
ep

epoch:146 loss:0.468977 accu:0.777344 gene_loss:1.160611
epoch:147 loss:0.534357 accu:0.714844 gene_loss:1.106809
epoch:148 loss:0.384121 accu:0.847656 gene_loss:1.480887
epoch:149 loss:1.121359 accu:0.613281 gene_loss:1.947453
epoch:150 loss:1.055746 accu:0.605469 gene_loss:0.664682
epoch:151 loss:0.876746 accu:0.570312 gene_loss:0.834355
epoch:152 loss:0.494320 accu:0.773438 gene_loss:1.414364
epoch:153 loss:0.804344 accu:0.683594 gene_loss:1.070187
epoch:154 loss:1.082852 accu:0.539062 gene_loss:0.479629
epoch:155 loss:0.643883 accu:0.628906 gene_loss:0.808145
epoch:156 loss:0.337604 accu:0.800781 gene_loss:1.230205
epoch:157 loss:0.359933 accu:0.902344 gene_loss:1.478685
epoch:158 loss:0.511833 accu:0.738281 gene_loss:0.845462
epoch:159 loss:0.536793 accu:0.683594 gene_loss:0.757973
epoch:160 loss:0.410480 accu:0.789062 gene_loss:1.036186
epoch:161 loss:0.386410 accu:0.796875 gene_loss:1.119257
epoch:162 loss:0.307988 accu:0.886719 gene_loss:1.232435
epoch:163 loss:1.124411 accu:0.

In [53]:
discriminator_i.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input1 (InputLayer)          (None, 28, 28, 1)         0         
_________________________________________________________________
discriminator (Sequential)   (None, 1)                 533505    
Total params: 533,505
Trainable params: 0
Non-trainable params: 533,505
_________________________________________________________________


In [54]:
generator_i.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input1 (InputLayer)          (None, 100)               0         
_________________________________________________________________
generator (Sequential)       (None, 28, 28, 1)         1097744   
Total params: 1,097,744
Trainable params: 1,095,184
Non-trainable params: 2,560
_________________________________________________________________


In [55]:
combined_model_i.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
z (InputLayer)               (None, 100)               0         
_________________________________________________________________
generator_Model (Model)      (None, 28, 28, 1)         1097744   
_________________________________________________________________
discriminator_Model (Model)  (None, 1)                 533505    
Total params: 1,631,249
Trainable params: 1,095,184
Non-trainable params: 536,065
_________________________________________________________________
